# NEURONAL LINKS😎️

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [2]:
labels = pd.read_csv('../../csv/train_labels.csv')
labels.head()

building_id  damage_grade
0       802906             3
1        28830             2
2        94947             3
3       590882             2
4       201944             3

In [3]:
values = pd.read_csv('../../csv/train_values.csv')
values.T

0      1      2       3       4       \
building_id                             802906  28830  94947  590882  201944   
geo_level_1_id                               6      8     21      22      11   
geo_level_2_id                             487    900    363     418     131   
geo_level_3_id                           12198   2812   8973   10694    1488   
count_floors_pre_eq                          2      2      2       2       3   
age                                         30     10     10      10      30   
area_percentage                              6      8      5       6       8   
height_percentage                            5      7      5       5       9   
land_surface_condition                       t      o      t       t       t   
foundation_type                              r      r      r       r       r   
roof_type                                    n      n      n       n       n   
ground_floor_type                            f      x      f       f       f   
other_floor_type                             q      q      x       x       x   
position                                     t      s      t       s       s   
plan_configuration                           d      d      d       d       d   
has_superstructure_adobe_mud                 1      0      0       0       1   
has_superstructure_mud_mortar_stone          1      1      1       1       0   
has_superstructure_stone_flag                0      0      0       0       0   
has_superstructure_cement_mortar_stone       0      0      0       0       0   
has_superstructure_mud_mortar_brick          0      0      0       0       0   
has_superstructure_cement_mortar_brick       0      0      0       0       0   
has_superstructure_timber                    0      0      0       1       0   
has_superstructure_bamboo                    0      0      0       1       0   
has_superstructure_rc_non_engineered         0      0      0       0       0   
has_superstructure_rc_engineered             0      0      0       0       0   
has_superstructure_other                     0      0      0       0       0   
legal_ownership_status                       v      v      v       v       v   
count_families                               1      1      1       1       1   
has_secondary_use                            0      0      0       0       0   
has_secondary_use_agriculture                0      0      0       0       0   
has_secondary_use_hotel                      0      0      0       0       0   
has_secondary_use_rental                     0      0      0       0       0   
has_secondary_use_institution                0      0      0       0       0   
has_secondary_use_school                     0      0      0       0       0   
has_secondary_use_industry                   0      0      0       0       0   
has_secondary_use_health_post                0      0      0       0       0   
has_secondary_use_gov_office                 0      0      0       0       0   
has_secondary_use_use_police                 0      0      0       0       0   
has_secondary_use_other                      0      0      0       0       0   

                                        5       6       7       8       \
building_id                             333020  728451  475515  441126   
geo_level_1_id                               8       9      20       0   
geo_level_2_id                             558     475     323     757   
geo_level_3_id                            6089   12066   12236    7219   
count_floors_pre_eq                          2       2       2       2   
age                                         10      25       0      15   
area_percentage                              9       3       8       8   
height_percentage                            5       4       6       6   
land_surface_condition                       t       n       t       t   
foundation_type                              r       r       w       r   
roof_type                

In [4]:
values["building_id"].count() == values["building_id"].drop_duplicates().count()

True

In [5]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")

In [6]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)

In [7]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)

In [8]:
important_values = values\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
important_values["geo_level_1_id"] = important_values["geo_level_1_id"].astype("category")


In [9]:
X_train, X_test, y_train, y_test = train_test_split(important_values.drop(columns = 'damage_grade'),
                                                    important_values['damage_grade'], test_size = 0.2, random_state = 123)

In [10]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)

In [11]:
import sklearn as sk
import sklearn.neural_network

In [29]:
neuronal = sk.neural_network.MLPClassifier(solver='adam', 
                                     activation = 'relu',
                                     learning_rate_init=0.001,
                                     learning_rate = 'adaptive',
                                     verbose=True,
                                     batch_size = 'auto')

neuronal.fit(X_train, y_train)

Iteration 1, loss = 5.25474738
Iteration 2, loss = 3.18594413
Iteration 3, loss = 2.88469088
Iteration 4, loss = 2.81965069
Iteration 5, loss = 2.44552548
Iteration 6, loss = 2.51948746
Iteration 7, loss = 2.60433442
Iteration 8, loss = 2.21060171
Iteration 9, loss = 2.26547626
Iteration 10, loss = 2.14678334
Iteration 11, loss = 2.19980556
Iteration 12, loss = 2.09647192
Iteration 13, loss = 2.11160082
Iteration 14, loss = 1.98182551
Iteration 15, loss = 2.08564199
Iteration 16, loss = 2.01294948
Iteration 17, loss = 1.87938322
Iteration 18, loss = 1.89419360
Iteration 19, loss = 1.73249508
Iteration 20, loss = 1.82456593
Iteration 21, loss = 1.86037611
Iteration 22, loss = 1.77363469
Iteration 23, loss = 1.58876646
Iteration 24, loss = 1.65221463
Iteration 25, loss = 1.53940986
Iteration 26, loss = 1.72265316
Iteration 27, loss = 1.44076744
Iteration 28, loss = 1.66661238
Iteration 29, loss = 1.32615156
Iteration 30, loss = 1.44589073
Iteration 31, loss = 1.35390101
Iteration 32, los

MLPClassifier(learning_rate='adaptive', verbose=True)

In [30]:
neuronal_pred = neuronal.predict(X_test)
f1_score(y_test, neuronal_pred, average='micro')

0.6651637535734157

ni me guardo el csv imaginate lo malo que es